In [5]:
import pandas as pd
import json
import os

### Definition of input folder

In [14]:
folder_path = "../study"
stats_path = folder_path + "/stats"

### Concatenation of all patient's metrics

In [20]:
## Read the list of subjects and for each subject do the tractography
dest_success = folder_path + "/subjects/subj_list.json"
with open(dest_success, 'r') as file:
    patient_list = json.load(file)

dfs = []
for p_code in patient_list:
    metric_folder = "%s/subjects/%s/dMRI/microstructure/%s_metrics.csv" % (folder_path, p_code, p_code)
    if not os.path.exists(metric_folder):
        print(metric_folder, "doesn't exists")
        continue
    dfs.append(pd.read_csv(metric_folder))

df = pd.concat(dfs)

if not os.path.isdir(stats_path):
    os.mkdir(stats_path)

df.to_csv("%s/dataset.csv" % stats_path, index=False)


../study/subjects/VNSLC_01/dMRI/microstructure/VNSLC_01_metrics.csv doesn't exists
../study/subjects/VNSLC_02/dMRI/microstructure/VNSLC_02_metrics.csv doesn't exists
../study/subjects/VNSLC_03/dMRI/microstructure/VNSLC_03_metrics.csv doesn't exists


## Analysis

In [21]:
df = pd.read_csv("%s/dataset.csv" % stats_path)

df

,right-thalamus-AntCingCtx_FA_mean,right-thalamus-AntCingCtx_FA_std,right-thalamus-AntCingCtx_FA_skew,right-thalamus-AntCingCtx_FA_kurt,right-sup-longi-fasci_FA_mean,right-sup-longi-fasci_FA_std,right-sup-longi-fasci_FA_skew,right-sup-longi-fasci_FA_kurt,left-thalamus-Insula_FA_mean,left-thalamus-Insula_FA_std,...,right-hippocampus_frac_csf_mf_skew,right-hippocampus_frac_csf_mf_kurt,right-amygdala_frac_csf_mf_mean,right-amygdala_frac_csf_mf_std,right-amygdala_frac_csf_mf_skew,right-amygdala_frac_csf_mf_kurt,right-accumbens-area_frac_csf_mf_mean,right-accumbens-area_frac_csf_mf_std,right-accumbens-area_frac_csf_mf_skew,right-accumbens-area_frac_csf_mf_kurt
0,0.473695,0.193161,-0.00008,0.003329,0.500848,0.153622,-0.001194,0.001566,0.44025,0.145429,...,0.04991,0.040012,0.137749,0.19797,0.024946,0.021099,0.076892,0.136489,0.014051,0.012932


-7.973509627576786e-05